<a href="https://colab.research.google.com/github/mirianbatista/vacinas-covid19/blob/master/test_change_size_request_elasticsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/spark-3.1.2-bin-hadoop3.2"

import findspark
findspark.init('spark-3.1.2-bin-hadoop3.2')

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import requests
from requests.auth import HTTPBasicAuth 
import pyspark.sql.functions as F

In [ ]:
spark = (SparkSession.builder.appName('Vacinas covid-19')
        .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.hadoop.fs.s3a.fast.upload", "true")
        .config("com.amazonaws.services.s3.enableV4", "true") 
        #.config('fs.s3a.access.key', access_key)
        #.config('fs.s3a.secret.key', secret_key)
        .getOrCreate())

22/06/01 12:13:22 WARN Utils: Your hostname, mirianbatista resolves to a loopback address: 127.0.1.1; using 192.168.0.14 instead (on interface wlp2s0)
22/06/01 12:13:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/06/01 12:13:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
r = requests.get('https://imunizacao-es.saude.gov.br/desc-imunizacao/_search?size=10000', auth=('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'))  
resp = r.json()

In [ ]:
r = requests.get('https://imunizacao-es.saude.gov.br/desc-imunizacao/_search?size=10000/_pit?keep_alive=1m', auth=('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'))  
resp = r.json()

In [ ]:
r = requests.get('https://imunizacao-es.saude.gov.br/desc-imunizacao/_search?size=10000/scroll=1m', auth=('imunizacao_public', 'qlto5t&7r_@+#Tlstigi'))  
resp = r.json()

In [ ]:
r

<Response [400]>

In [ ]:
r.text

'{"error":{"root_cause":[{"type":"illegal_argument_exception","reason":"Failed to parse int parameter [size] with value [10000/scroll=1m]"}],"type":"illegal_argument_exception","reason":"Failed to parse int parameter [size] with value [10000/scroll=1m]","caused_by":{"type":"number_format_exception","reason":"For input string: \\"10000/scroll=1m\\""}},"status":400}'

In [ ]:
rdd = spark.sparkContext.parallelize([r.text])

df = spark.read.json(rdd)

In [ ]:
df.show()

+--------------------+------+
|               error|status|
+--------------------+------+
|{{For input strin...|   400|
+--------------------+------+



In [ ]:
rdd

ParallelCollectionRDD[64] at readRDDFromFile at PythonRDD.scala:274

In [ ]:
q = df[['hits']]
df_final = (q.withColumn("teste_source", F.col("hits")["hits"])
             .withColumn('exploded_arr',explode('teste_source'))
             .withColumn("source_exploded", F.col("exploded_arr")["_source"]))

In [ ]:
df_finalmente = (df_final.withColumn("paciente_id",df_final.source_exploded["paciente_id"])
                         .withColumn("paciente_dataNascimento",df_final.source_exploded["paciente_dataNascimento"])
                         .withColumn("paciente_racaCor_valor",df_final.source_exploded["paciente_racaCor_valor"])
                         .withColumn("paciente_racaCor_codigo",df_final.source_exploded["paciente_racaCor_codigo"])
                         .withColumn("paciente_endereco_nmMunicipio",df_final.source_exploded["paciente_endereco_nmMunicipio"])
                         .withColumn("paciente_idade",df_final.source_exploded["paciente_idade"])
                         .withColumn("paciente_endereco_coIbgeMunicipio",df_final.source_exploded["paciente_endereco_coIbgeMunicipio"])
                         .withColumn("vacina_dataAplicacao",df_final.source_exploded["vacina_dataAplicacao"])
                         .drop("hits").drop("teste_source").drop("exploded_arr").drop("source_exploded"))
df_finalmente.show(3)

22/06/01 15:43:39 WARN TaskSetManager: Stage 24 contains a task of very large size (16323 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----------------------+----------------------+-----------------------+-----------------------------+--------------+---------------------------------+--------------------+
|         paciente_id|paciente_dataNascimento|paciente_racaCor_valor|paciente_racaCor_codigo|paciente_endereco_nmMunicipio|paciente_idade|paciente_endereco_coIbgeMunicipio|vacina_dataAplicacao|
+--------------------+-----------------------+----------------------+-----------------------+-----------------------------+--------------+---------------------------------+--------------------+
|bfa49f8a3e3fae543...|             2002-12-15|                BRANCA|                     01|                  PASSO FUNDO|            19|                           431410|2022-05-24T00:00:...|
|527d27d520c2d84b2...|             2001-04-06|                 PRETA|                     02|         GOVERNADOR VALADARES|            21|                           312770|2022-05-28T00:00:...|
|7294bb82988f6d329...|        

In [ ]:
df_finalmente.printSchema()

root
 |-- paciente_id: string (nullable = true)
 |-- paciente_dataNascimento: string (nullable = true)
 |-- paciente_racaCor_valor: string (nullable = true)
 |-- paciente_racaCor_codigo: string (nullable = true)
 |-- paciente_endereco_nmMunicipio: string (nullable = true)
 |-- paciente_idade: long (nullable = true)
 |-- paciente_endereco_coIbgeMunicipio: string (nullable = true)
 |-- vacina_dataAplicacao: string (nullable = true)



In [ ]:
df_value_counts = df_finalmente.groupBy("paciente_id").count().orderBy(col("count").desc())
df_value_counts.show()

22/06/01 15:43:39 WARN TaskSetManager: Stage 25 contains a task of very large size (16323 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+-----+
|         paciente_id|count|
+--------------------+-----+
|49da3de973d4bec46...|    2|
|1480e5dfebf72daa7...|    2|
|0e075efb9f0b368b8...|    2|
|0325f6ab1dbc9efdf...|    2|
|c11b7bda930531b4f...|    2|
|06662be725b3c1bea...|    2|
|6488ad480875b66ce...|    2|
|b34b2cc5791fe37dc...|    1|
|4e2ccb0e0707347d6...|    1|
|8e385f21622b2bf55...|    1|
|b13e1120d9132c6c6...|    1|
|c1e0005fa1392e932...|    1|
|fc4add40920a61f63...|    1|
|fe6316992311742fb...|    1|
|f54ccbf40f0325320...|    1|
|cac441d5a969d3dcb...|    1|
|850ba320ca689cbc7...|    1|
|d38462cdfc714ad8a...|    1|
|8aff4ce427f5bf77a...|    1|
|4857109109679ccdf...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
df_vacinados = df_value_counts.groupBy().pivot("count").count().withColumnRenamed("1","qtd_1_dose").withColumnRenamed("2","qtd_2_doses")
df_vacinados.show()                                                       

22/06/01 16:00:33 WARN TaskSetManager: Stage 63 contains a task of very large size (16323 KiB). The maximum recommended task size is 1000 KiB.
22/06/01 16:00:34 WARN TaskSetManager: Stage 67 contains a task of very large size (16323 KiB). The maximum recommended task size is 1000 KiB.


+----------+-----------+
|qtd_1_dose|qtd_2_doses|
+----------+-----------+
|      9986|          7|
+----------+-----------+



In [ ]:
url = 'https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SIPNI/COVID/completo/part-00000-3da2bebd-9aac-4f31-9e00-8e48af97d46d-c000.csv'

In [ ]:
import pandas as pd

df_aa = pd.read_csv(url)

MemoryError: 

In [ ]:
from pyspark import SparkFiles
spark.sparkContext.addFile('https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SIPNI/COVID/completo/part-00000-3da2bebd-9aac-4f31-9e00-8e48af97d46d-c000.csv')
df_teste_csv = spark.read.csv(SparkFiles.get("part-00000-3da2bebd-9aac-4f31-9e00-8e48af97d46d-c000.csv"), header=True)
df_teste_csv.show()

22/06/01 16:19:01 WARN SparkContext: The path https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SIPNI/COVID/completo/part-00000-3da2bebd-9aac-4f31-9e00-8e48af97d46d-c000.csv has been added already. Overwriting of added paths is not supported in the current version.


AnalysisException: Path does not exist: file:/tmp/spark-c559d328-b85c-45ba-ac5c-6d8911523e18/userFiles-2933d55c-5394-411e-a767-c994335070bf/part-00000-3da2bebd-9aac-4f31-9e00-8e48af97d46d-c000.csv